# Structured Outputs

## JSON Mode

JSON mode will ensure that the model outputs valid JSON.the JSON schema should be provided within the prompt in one of the messages. To enable JSON mode `response_format` needs to be set to `{ "type": "json_object" }`

## Example:
Let's use an use case such as news summarization to see how JSON mode can be used.
[News Article](https://www.forbes.com/sites/tylerroush/2024/09/12/fda-approves-first-hearing-aid-software-for-apples-airpods/)

In [10]:
article_content = """
FDA Approves First Hearing Aid Software For Apple's AirPods

Ty Roush
Forbes Staff
Ty Roush is a breaking news reporter based in New York City.

Sep 12, 2024,12:55pm EDT

The Food and Drug Administration on Thursday allowed the first over-the-counter hearing aid software device to hit the market, a feature Apple announced earlier this week would be compatible with its latest earbuds to assist consumers they say may be unaware they have hearing loss.

Key Facts
The FDA approved the software device, “Hearing Aid Feature,” which the agency said could be installed on Apple's AirPods Pro headphones to amplify sounds for anyone 18 or older with mild to moderate hearing impairment.

Apple announced Monday its upcoming Apple AirPods Pro 2 could “transform” into over-the-counter hearing aids through an upcoming software update that would increase some sounds in real-time, including voices or other sounds in a user's environment.

Apple said it expects to include the hearing aid software in an update in the fall.

More than 30 million. That's how many American adults have reported some degree of hearing loss, which can be caused by aging, exposure to loud noises and certain medical conditions, among other factors, the FDA said.

Key Background
The Hearing Aid Feature is new software intended to expand access to hearing support, the FDA said. Access was previously much more limited, the FDA noted, before the agency announced new regulations in 2022 allowing consumers with hearing impairments to purchase hearing aids directly from stores or online retailers without a medical exam or prescription. Apple has advertised the software as a benefit for “millions of people” unaware they are living with hearing loss, adding users can receive hearing tests through their iPhones. The software was evaluated in a study involving 118 participants with mild to moderate hearing loss, indicating patients who used the software received similar benefits to those who used traditional hearing aids, the FDA said.

Crucial Quote
Michelle Tarver, director of the FDA's Center for Devices and Radiological Health, said in a statement the software's authorization “advances the availability, accessibility and acceptability” of hearing support for adults.

"""

In [11]:
response_format = {
    "title": "",
    "author": "",
    "date": "",
    "summary": "",
    "sentiment": "",
    "keywords": [],
    "main_company": "",
    "other_orgs": "list of other organizations mentioned in the article",
}

In [12]:
from dotenv import load_dotenv
from openai import OpenAI
import json

load_dotenv("../../.env")

client = OpenAI()

PROMPT_TEMPLATE = """
You are a news summarizer. Your task is to summarize the following article:

## Content:
{content}

## Response Format:
{response_format}

## Response:

"""

def complete(article_content, response_format):
    prompt = PROMPT_TEMPLATE.format(
        content=article_content, 
        response_format=response_format
    )

    response = client.chat.completions.create(
        model="gpt-4o-mini",
        response_format={"type": "json_object"},
        messages=[
            {"role": "system", "content": "you are an assistant that responds in JSON"},
            {"role": "user", "content": prompt}
        ]
    )

    return json.loads(response.choices[0].message.content)

In [13]:
response = complete(article_content, response_format)
response

{'title': "FDA Approves First Hearing Aid Software For Apple's AirPods",
 'author': 'Ty Roush',
 'date': 'Sep 12, 2024',
 'summary': "The FDA has approved a new software feature that allows Apple's AirPods Pro to function as over-the-counter hearing aids for adults with mild to moderate hearing impairment. This software, set to be released in a fall update, will amplify sounds in real-time, making it easier for users to hear voices and environmental sounds. The approval aims to provide wider access to hearing support, particularly as over 30 million American adults experience some level of hearing loss.",
 'sentiment': 'positive',
 'keywords': ['FDA',
  'Hearing Aid Feature',
  'Apple AirPods Pro',
  'hearing loss',
  'software update',
  'accessibility'],
 'main_company': 'Apple',
 'other_orgs': ['FDA']}

## Structured Outputs using Pydantic

This is a new and better method to ensure generattion of structured Outputs. 

Pydantic is a data validation and settings management using Python type annotations. Pydantic enforces type hints at runtime, and provides user friendly errors when data is invalid.

Pydantic models can be used to define the schema of the output. The model can be used to validate the output and ensure that the output is in the correct format.

In [20]:
from pydantic import BaseModel
from enum import Enum

class Sentiment(str, Enum):
    very_negative = "very negative"
    negative = "negative"
    neutral = "neutral"
    positive = "positive"
    very_positive = "very positive"

class Article(BaseModel):
    title: str
    author: str
    date: str
    summary: str
    sentiment: Sentiment
    keywords: list[str]
    main_company: str
    other_orgs: list[str]

### There are some changes from compared to the normal way of completions endpoint:

1. Use `client.beta.chat.completions.parse()` instead of `client.chat.completions.create()`
2. Set `response_format` to the pydantic schema defined.
3. Use the `response.choices[0].message.parsed` to get the output in the desired format.

In [23]:
def complete(article_content, response_format):
    response = client.beta.chat.completions.parse(
        model="gpt-4o-mini",
        messages=[
            {"role": "system", "content": "you are a new summarizer that will extract key details from news article content."},
            {"role": "user", "content": article_content}
        ],
        response_format=response_format
    )

    return response.choices[0].message.parsed

In [25]:
article = complete(article_content,Article)
article.model_dump()

{'title': "FDA Approves First Hearing Aid Software For Apple's AirPods",
 'author': 'Ty Roush',
 'date': 'Sep 12, 2024,12:55pm EDT',
 'summary': 'The FDA has approved an over-the-counter hearing aid software that can be installed on Apple AirPods Pro headphones, aimed at helping adults aged 18 and older with mild to moderate hearing impairments. This feature, expected to roll out in a fall update, allows users to amplify real-time sounds. Over 30 million American adults have reported some degree of hearing loss, and this new regulation aims to improve access to hearing aids.',
 'sentiment': <Sentiment.positive: 'positive'>,
 'keywords': ['FDA',
  'hearing aid',
  'Apple',
  'AirPods Pro',
  'hearing loss',
  'technology'],
 'main_company': 'Apple',
 'other_orgs': ['FDA']}